<a href="https://colab.research.google.com/github/forest1988/colaboratory/blob/main/prophetnet_seq2seqtrainer_finetuning_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ProphetNet fine-tuning with Seq2Seq Trainer

- Fine-tuning experiment using ProphetNet in HuggingFace Transformer with Seq2Seq Trainer

- Fine-tuning for summarization task
  - dataset: xsum or cnn-dm

In [ ]:
!nvidia-smi

Tue Nov 17 14:56:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch

In [ ]:
torch.__version__

'1.7.0+cu101'

In [ ]:
!git clone https://github.com/huggingface/transformers.git

fatal: destination path 'transformers' already exists and is not an empty directory.


In [ ]:
%cd transformers/

/content/transformers


In [ ]:
# !git log | grep "prepare_seq2seq_batch function to ProphetNet"

In [ ]:
!pip install -e .

Obtaining file:///content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Found existing installation: transformers 4.0.0.dev0
    Can't uninstall 'transformers'. No files were found to uninstall.
  Running setup.py develop for transformers


In [ ]:
%cd /content/transformers/examples/seq2seq/

/content/transformers/examples/seq2seq


In [ ]:
# to avoid the "ModuleNotFoundError" 
import sys
print(sys.path)
sys.path.append('/content/transformers/src')
print(sys.path)

['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/content/transformers/src', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython']
['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/content/transformers/src', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', '/content/transformers/src']


In [ ]:
# If "ModuleNotFoundError" occurs in this cell, please re-run the runtime. 
# (The first time Colab runtime runs, it seems the sys.path is not correctly updated when we use `pip install -e .`)

import transformers

In [ ]:
transformers.__version__

'4.0.0-dev'

In [ ]:
!pip install cookiecutter
!transformers-cli env

2020-11-17 14:56:31.362609: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
2020-11-17 14:56:33.235745: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2200000000 Hz
2020-11-17 14:56:33.235991: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7bbafc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-17 14:56:33.236027: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-11-17 14:56:33.243232: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-17 14:56:33.353054: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so retu

## dataset 

In [ ]:
!wget https://cdn-datasets.huggingface.co/summarization/xsum.tar.gz
!tar -xzvf xsum.tar.gz

--2020-11-17 14:56:40--  https://cdn-datasets.huggingface.co/summarization/xsum.tar.gz
Resolving cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)... 99.84.181.104, 99.84.181.99, 99.84.181.50, ...
Connecting to cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)|99.84.181.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 204844092 (195M) [application/x-tar]
Saving to: ‘xsum.tar.gz.1’

xsum.tar.gz.1       100%[===================>] 195.35M   234MB/s    in 0.8s    

2020-11-17 14:56:41 (234 MB/s) - ‘xsum.tar.gz.1’ saved [204844092/204844092]

xsum/
xsum/train.target
xsum/train.source
xsum/val.source
xsum/val.target
xsum/test.source
xsum/test.target


In [ ]:
import os
os.environ['XSUM_DIR']=os.path.join(os.getcwd(), 'xsum')

In [ ]:
!echo $XSUM_DIR

/content/transformers/examples/seq2seq/xsum


In [ ]:
!ls $XSUM_DIR

test.source  test.target  train.source	train.target  val.source  val.target


In [ ]:
!wc -l $XSUM_DIR/*

    11333 /content/transformers/examples/seq2seq/xsum/test.source
    11333 /content/transformers/examples/seq2seq/xsum/test.target
   204017 /content/transformers/examples/seq2seq/xsum/train.source
   204017 /content/transformers/examples/seq2seq/xsum/train.target
    11327 /content/transformers/examples/seq2seq/xsum/val.source
    11327 /content/transformers/examples/seq2seq/xsum/val.target
   453354 total


In [ ]:
!wget https://cdn-datasets.huggingface.co/summarization/cnn_dm_v2.tgz
!tar -xzvf cnn_dm_v2.tgz  # empty lines removed
!mv cnn_cln cnn_dm

os.environ['CNN_DIR']=os.path.join(os.getcwd(), 'cnn_dm')

--2020-11-17 14:56:47--  https://cdn-datasets.huggingface.co/summarization/cnn_dm_v2.tgz
Resolving cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)... 99.84.181.99, 99.84.181.83, 99.84.181.50, ...
Connecting to cdn-datasets.huggingface.co (cdn-datasets.huggingface.co)|99.84.181.99|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 514271123 (490M) [application/x-tar]
Saving to: ‘cnn_dm_v2.tgz.1’

cnn_dm_v2.tgz.1     100%[===================>] 490.45M   274MB/s    in 1.8s    

2020-11-17 14:56:49 (274 MB/s) - ‘cnn_dm_v2.tgz.1’ saved [514271123/514271123]

cnn_cln/
cnn_cln/train.source
cnn_cln/train.target
cnn_cln/test.target
cnn_cln/test.source
cnn_cln/val.source
cnn_cln/val.target


## Install required package

In [ ]:
%%capture
!pip install gitpython
!pip install rouge_score
!pip install sacrebleu

## Fine-tune ProphetNet

- Due to Colab's time limit (12 hours), about one-tenth of the dataset is used here.

In [ ]:
!python finetune_trainer.py \
    --learning_rate=3e-5 \
    --fp16 \
    --do_train --do_eval --do_predict --evaluate_during_training \
    --predict_with_generate \
    --max_source_length 510 \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 2 \
    --n_train 20400 \
    --n_val 100 \
    --model_name_or_path microsoft/prophetnet-large-uncased \
    --data_dir $XSUM_DIR \
    --output_dir prophetnet_large_uncased_xsum \
    --save_steps 10000 \
    --save_total_limit 5 \
    --overwrite_output_dir 

2020-11-17 14:57:23.833868: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/content/transformers/src/transformers/training_args.py:347: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,
11/17/2020 14:57:25 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: True
11/17/2020 14:57:25 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='prophetnet_large_uncased_xsum', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=True, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=2, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps

In [21]:
from google.colab import files
import glob

output_files = glob.glob("/content/transformers/examples/seq2seq/prophetnet_large_uncased_xsum/*")

for filename in output_files:
  if "results" in filename or "generation" in filename:
    files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Run BART-base

In [ ]:
!python finetune_trainer.py \
    --learning_rate=3e-5 \
    --fp16 \
    --do_train --do_eval --do_predict --evaluate_during_training \
    --predict_with_generate \
    --max_source_length 510 \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 2 \
    --n_train 20400 \
    --n_val 100 \
    --model_name_or_path facebook/bart-base \
    --data_dir $XSUM_DIR \
    --output_dir bart_base_xsum \
    --save_steps 10000 \
    --save_total_limit 1 \
    --overwrite_output_dir 